In [2]:
import numpy as np
import openpyxl
import string
import datetime
from scipy import interpolate
import calendar
import csv
import os

# Country

In [3]:
# US, EA, JP, UK. 
Country   = 'US'

In [4]:
# def write_to_csv(idx, idx1, val, val1, sub_datenum, name):
#     daily_values_data    = map(list, zip(*val))
#     daily_values_data1   = map(list, zip(*val1))

#     daily_date_index     = list(idx)
#     daily_date_index[:]  = [x - sub_datenum for x in idx]

#     daily_date_index1    = list(idx1)
#     daily_date_index1[:] = [x - sub_datenum for x in idx1]
    
#     for i in range(len(daily_values_data)):
#         daily_values_data[i].insert(0, daily_date_index[i])
#     for i in range(len(daily_values_data1)):
#         daily_values_data1[i].insert(0, daily_date_index1[i])
        
#     with open(name+'.csv', 'wb') as csvfile:
#         spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_NONNUMERIC)
#         spamwriter.writerows(daily_values_data1)
#         spamwriter.writerows(daily_values_data)

In [5]:
def write_to_csv_updated(idx, idx1, val, val1, sub_datenum, name):
    daily_values_data    = []
    daily_values_data1   = []

    daily_date_index     = list(idx)
    daily_date_index[:]  = [x - sub_datenum for x in idx]

    daily_date_index1    = list(idx1)
    daily_date_index1[:] = [x - sub_datenum for x in idx1]
    
    for index, i in enumerate(map(list, zip(*val))):
        daily_values_data.append([daily_date_index[index]] + i)
        
    for index, i in enumerate(map(list, zip(*val1))):
        daily_values_data1.append([daily_date_index1[index]] + i)
        
    print(np.array(daily_values_data).shape)
    print(np.array(daily_values_data1).shape)
        
    with open(name+'.csv', 'w') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_NONNUMERIC)
        spamwriter.writerows(map(list, zip(*np.array(daily_values_data1).T.tolist())))
        spamwriter.writerows(map(list, zip(*np.array(daily_values_data).T.tolist())))

In [6]:
def filter_data_index(a, thr, method):
    if method is 'ge':
        index = list([idx for idx,i in enumerate(a) if i < thr])
    elif method is 'le':
        index = list([idx for idx,i in enumerate(a) if i > thr])
    elif method is 'l':
        index = list([idx for idx,i in enumerate(a) if i >= thr])
    elif method is 'g':
        index = list([idx for idx,i in enumerate(a) if i <= thr])
    return index

In [7]:
def filter_data(a, index, dim):
    b = list(a)
    if dim > 1:
        for i in range(dim):
            b[i] = list(a[i])
            for idx in reversed(index):
                del b[i][idx]
    else:
        for idx in reversed(index):
            del b[idx]
    return b

In [8]:
list1     = list(string.ascii_uppercase)
list2     = ['A'+x for x in list1]

list1     = list1 + list2

datelist  = list1[0:36:3]
valuelist = list1[1:36:3]

In [9]:
list1[0:36:3]

['A', 'D', 'G', 'J', 'M', 'P', 'S', 'V', 'Y', 'AB', 'AE', 'AH']

In [10]:
list1[1:36:3]

['B', 'E', 'H', 'K', 'N', 'Q', 'T', 'W', 'Z', 'AC', 'AF', 'AI']

In [11]:

CurveType = 'OIS'
PathName  = os.getcwd()
ExcelName = os.path.join(PathName, 'A_'+Country+'_All_Data_Bloomberg.xlsx')
if CurveType is 'OIS':
    ExcelSheetName = 'D. Live OIS data'
if CurveType is 'Govt':
    ExcelSheetName = 'D. Live Govt data'
    
Maturities = [0.25, 0.5, 1, 2, 3, 4, 5, 7, 10, 15, 20, 30]

In [12]:
ExcelName

'/Users/akshay/Documents/Berkeley/Industry Projects/BlackRock/Leo_Krippner_SSR-TestAkshayJoshi/A_US_All_Data_Bloomberg.xlsx'

In [13]:
%%time
wb          = openpyxl.load_workbook(ExcelName)
sheet_names = wb.sheetnames
sheet       = wb[ExcelSheetName]
datenum     = [[] for i in range(len(datelist))]
values_data = [[] for i in range(len(datelist))]

CPU times: user 3.87 s, sys: 43.7 ms, total: 3.91 s
Wall time: 3.92 s


In [14]:
wb.sheetnames

['D. Live OIS data', 'D. Live Govt data']

In [15]:
wb['D. Live OIS data']

<Worksheet "D. Live OIS data">

In [16]:
%%time

for col_num in range(len(datelist)):
    DateGen = sheet[datelist[col_num] + '8':datelist[col_num]+'8000']
    for i in DateGen:
        if i[0].value is not None:
            a = i[0].value.date().toordinal()
            datenum[col_num].append(a)
    ValueGen = sheet[valuelist[col_num]+'8':valuelist[col_num]+'8000']
    
    for i in ValueGen:
        if i[0].value is not None:
            a=i[0].value
            values_data[col_num].append(a)

CPU times: user 908 ms, sys: 16.9 ms, total: 925 ms
Wall time: 926 ms


In [17]:
common_datenum=list(datenum[0])
for i in range(1,len(datenum)):
    common_datenum = list(set(common_datenum)&set(datenum[i]))

In [18]:
common_datenum.sort()
for i in range(len(datenum)):
    diff_dates = list(set(common_datenum) ^ set(datenum[i]))
    for j in diff_dates:
        index  = datenum[i].index(j)
        print(str(j) + ' date num at ' + str(index) + 'removed') 
        value_to_remove = values_data[i][index]
        values_data[i].remove(value_to_remove)

In [19]:
ref_friday         = datetime.date(2013, 5,24).toordinal()
weeks_to_step_back = (ref_friday - common_datenum[0])/7
first_friday       = int(ref_friday - (weeks_to_step_back * 7))

In [20]:
week_date_index = range(first_friday, common_datenum[-1],7)

In [21]:
week_values_data = [[] for i in range(len(datelist))]
for i in range(len(datelist)):
    f = interpolate.interp1d(common_datenum, values_data[i], 'zero', bounds_error = False)
    week_values_data[i] = list(f(week_date_index))

In [22]:
month_date_index = []
first_date       = datetime.date.fromordinal(common_datenum[0])
last_date        = datetime.date.fromordinal(common_datenum[-1])

first_year       = first_date.year
last_year        = last_date.year

for i in range(first_date.month , 13, 1):
    (first_week_day,last_day_of_month) = calendar.monthrange(first_year,i)
    month_date_index.append(datetime.date(first_year, i, last_day_of_month).toordinal())
    
print(len(month_date_index))
    
for i in range(first_year + 1, last_year, 1):
    for month in range(1, 13, 1):
        (first_week_day,last_day_of_month) = calendar.monthrange(i,month)
        month_date_index.append(datetime.date(i,month,last_day_of_month).toordinal())
        
print(len(month_date_index))
        
for i in range(1,last_date.month + 1,1):
    (first_week_day,last_day_of_month) = calendar.monthrange(last_year,i)
    month_date_index.append(datetime.date(last_year,i,last_day_of_month).toordinal())
    
print(len(month_date_index))

12
168
177


In [23]:
month_values_data = [[] for i in range(len(datelist))]
for i in range(len(datelist)):
    f = interpolate.interp1d(common_datenum, values_data[i],'zero',bounds_error=False)
    month_values_data[i] = list(f(month_date_index))

In [24]:
BusinessDayTimestamp = float(common_datenum[-1]-common_datenum[0]+1)/(len(common_datenum)*365.25)

In [25]:
sheet        = wb['D. Live Govt data']
datenum1     = [[] for i in range(len(datelist))]
values_data1 = [[] for i in range(len(datelist))]

for col_num in range(len(datelist)):
    DateGen = sheet[datelist[col_num]+'8':datelist[col_num]+'8000']
    for i in DateGen:
        if i[0].value is not None:
            a = i[0].value.date().toordinal()
            datenum1[col_num].append(a)

    ValueGen = sheet[valuelist[col_num]+'8':valuelist[col_num]+'8000']

    for i in ValueGen:
        if i[0].value is not None:
            a = i[0].value
            values_data1[col_num].append(a)

common_datenum1 = list(datenum1[0])
for i in range(1,len(datenum1)):
    common_datenum1 = list(set(common_datenum1)&set(datenum1[i]))

common_datenum1.sort()

for i in range(len(datenum1)):
    diff_dates=list(set(common_datenum1) ^ set(datenum1[i]))
    for j in diff_dates:
        index=datenum1[i].index(j)
        print(str(j)+' date num at '+str(index)+'removed') 
        value_to_remove = values_data1[i][index]
        values_data1[i].remove(value_to_remove)

736815 date num at 6087removed
736816 date num at 6088removed
736815 date num at 6087removed
736816 date num at 6088removed
736815 date num at 6087removed
736816 date num at 6088removed
736815 date num at 6087removed
736816 date num at 6088removed
736815 date num at 6087removed
736816 date num at 6088removed
736815 date num at 6087removed
736816 date num at 6088removed
736815 date num at 6087removed
736816 date num at 6088removed
736815 date num at 6087removed
736816 date num at 6088removed
736815 date num at 6087removed
736816 date num at 6088removed
736815 date num at 6087removed
736816 date num at 6088removed


In [26]:
ref_friday=datetime.date(2013, 5, 24).toordinal()
weeks_to_step_back = (ref_friday - common_datenum1[0])/7
first_friday = int(ref_friday - (weeks_to_step_back*7))

week_date_index1 = range(first_friday,common_datenum1[-1],7)
week_values_data1=[[] for i in range(len(datelist))]
for i in range(len(datelist)):
    f=interpolate.interp1d(common_datenum1,values_data1[i],'zero',bounds_error=False)
    week_values_data1[i] = list(f(week_date_index1))

month_date_index1=[]
first_date=datetime.date.fromordinal(common_datenum1[0])
last_date=datetime.date.fromordinal(common_datenum1[-1])
first_year = first_date.year
last_year = last_date.year

for i in range(first_date.month,13,1):
    (first_week_day,last_day_of_month)=calendar.monthrange(first_year,i)
    month_date_index1.append(datetime.date(first_year,i,last_day_of_month).toordinal())
for i in range(first_year+1,last_year,1):
    for month in range(1,13,1):
        (first_week_day,last_day_of_month)=calendar.monthrange(i,month)
        month_date_index1.append(datetime.date(i,month,last_day_of_month).toordinal())
for i in range(1,last_date.month+1,1):
    (first_week_day,last_day_of_month)=calendar.monthrange(last_year,i)
    month_date_index1.append(datetime.date(last_year,i,last_day_of_month).toordinal())

month_values_data1=[[] for i in range(len(datelist))]
for i in range(len(datelist)):
    f=interpolate.interp1d(common_datenum1,values_data1[i],'zero',bounds_error=False)
    month_values_data1[i] = list(f(month_date_index1))

BusinessDayTimestamp1 = float(common_datenum1[-1]-common_datenum1[0]+1)/(len(common_datenum1)*365.25)

In [27]:
sub_datenum = datetime.date(1899,12,30).toordinal()

if Country == 'JP':
    firstday = datetime.date(2009,8,6).toordinal()
else:
    firstday = common_datenum[0]

lastday = firstday - 1

In [28]:
datetime.datetime.fromordinal(firstday), datetime.datetime.fromordinal(lastday)

(datetime.datetime(2006, 1, 4, 0, 0), datetime.datetime(2006, 1, 3, 0, 0))

In [29]:
index_to_del      = filter_data_index(common_datenum, firstday, 'ge')
new_daily_index   = filter_data(common_datenum, index_to_del, 1)
new_daily_values  = filter_data(values_data, index_to_del, len(values_data))

index_to_del      = filter_data_index(common_datenum1,lastday,'le')
new_daily_index1  = filter_data(common_datenum1,index_to_del,1)
new_daily_values1 = filter_data(values_data1,index_to_del,len(values_data1))

write_to_csv_updated(new_daily_index, new_daily_index1, new_daily_values, new_daily_values1, sub_datenum, Country+'_Daily')

index_to_del=filter_data_index(week_date_index,firstday,'ge')
new_daily_index=filter_data(week_date_index,index_to_del,1)
new_daily_values=filter_data(week_values_data,index_to_del,len(week_values_data))

index_to_del=filter_data_index(week_date_index1,lastday,'le')
new_daily_index1=filter_data(week_date_index1,index_to_del,1)
new_daily_values1=filter_data(week_values_data1,index_to_del,len(week_values_data1))

write_to_csv_updated(new_daily_index,new_daily_index1,new_daily_values,new_daily_values1,sub_datenum,Country+'_Weekly')

index_to_del=filter_data_index(month_date_index,firstday,'ge')
new_daily_index=filter_data(month_date_index,index_to_del,1)
new_daily_values=filter_data(month_values_data,index_to_del,len(month_values_data))

index_to_del=filter_data_index(month_date_index1,lastday,'le')
new_daily_index1=filter_data(month_date_index1,index_to_del,1)
new_daily_values1=filter_data(month_values_data1,index_to_del,len(month_values_data1))

write_to_csv_updated(new_daily_index,new_daily_index1,new_daily_values,new_daily_values1,sub_datenum,Country+'_Monthly')

(3836, 13)
(2873, 13)
(767, 13)
(575, 13)
(177, 13)
(133, 13)
